# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

B. Saha  ->  B. Saha  |  ['B. Saha']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Linz  ->  H. Linz  |  ['H. Linz']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 148 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.05376


extracting tarball to tmp_2410.05376...

 done.


Found 36 bibliographic references in tmp_2410.05376/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2410.05397


...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.06730


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2410.06697/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'S8_Appendix' from 'tmp_2410.06697/S8_Appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'S6_7_Acknowledgment' from 'tmp_2410.06697/S6_7_Acknowledgment.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/si

extracting tarball to tmp_2410.06730... done.
Retrieving document from  https://arxiv.org/e-print/2410.06836


extracting tarball to tmp_2410.06836...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.05376-b31b1b.svg)](https://arxiv.org/abs/2410.05376) | **Quantifying the impact of AGN feedback on the large-scale matter distribution using two- and three-point statistics**  |
|| <mark>B. Saha</mark>, S. Bose |
|*Appeared on*| *2024-10-10*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**|            Feedback from active galactic nuclei (AGN) plays a critical role in shaping the matter distribution on scales comparable to and larger than individual galaxies. Upcoming surveys such as $\textit{Euclid}$ and LSST aim to precisely quantify the matter distribution on cosmological scales, making a detailed understanding of AGN feedback effects essential. Hydrodynamical simulations provide an informative framework for studying these effects, in particular by allowing us to vary the parameters that determine the strength of these feedback processes and, consequently, to predict their corresponding impact on the large-scale matter distribution. We use the EAGLE simulations to explore how changes in subgrid viscosity and AGN heating temperature affect the matter distribution, quantified via 2- and 3-point correlation functions, as well as higher order cumulants of the matter distribution. We find that varying viscosity has a small impact ($\approx 10\%$) on scales larger than $1 h^{-1}$ Mpc, while changes to the AGN heating temperature lead to substantial differences, with up to $70\%$ variation in gas clustering on small scales ($\lesssim 1 h^{-1}$ Mpc). By examining the suppression of the power spectrum as a function of time, we identify the redshift range $z = 1.5 - 1$ as a key epoch where AGN feedback begins to dominate in these simulations. The 3-point function provides complementary insight to the more familiar 2-point statistics, and shows more pronounced variations between models on the scale of individual haloes. On the other hand, we find that effects on even larger scales are largely comparable.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.05397-b31b1b.svg)](https://arxiv.org/abs/2410.05397) | **Polycyclic Aromatic Hydrocarbon and CO(2-1) Emission at 50-150 pc Scales in 66 Nearby Galaxies**  |
|| R. Chown, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *21 pages, 4 figures, 3 tables. Submitted to ApJ*|
|**Abstract**|            Combining Atacama Large Millimeter/sub-millimeter Array CO(2-1) mapping and JWST near- and mid-infrared imaging, we characterize the relationship between CO(2-1) and polycyclic aromatic hydrocarbon (PAH) emission at ~100 pc resolution in 66 nearby star-forming galaxies, expanding the sample size from previous ~100 pc resolution studies by more than an order of magnitude. Focusing on regions of galaxies where most of the gas is likely to be molecular, we find strong correlations between CO(2-1) and 3.3 micron, 7.7 micron, and 11.3 micron PAH emission, estimated from JWST's F335M, F770W, and F1130W filters. We derive power law relations between CO(2-1) and PAH emission, which have indices in the range 0.8-1.2, implying relatively weak variations in the observed CO-to-PAH ratios across the regions that we study. We find that CO-to-PAH ratios and scaling relationships near HII regions are similar to those in diffuse sight lines. The main difference between the two types of regions is that sight lines near HII regions show higher intensities in all tracers. Galaxy centers, on the other hand, show higher overall intensities and enhanced CO-to-PAH ratios compared to galaxy disks. Individual galaxies show 0.19 dex scatter in the normalization of CO at fixed I_PAH and this normalization anti-correlates with specific star formation rate (SFR/M*) and correlates with stellar mass. We provide a prescription that accounts for these galaxy-to-galaxy variations and represents our best current empirical predictor to estimate CO(2-1) intensity from PAH emission, which allows one to take advantage of JWST's excellent sensitivity and resolution to trace cold gas.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.05761-b31b1b.svg)](https://arxiv.org/abs/2410.05761) | **Relatively young thick discs in low-mass star-forming late-type galaxies**  |
|| N. Sattler, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *20 pages, 9 figures, submitted to A&A*|
|**Abstract**|            We aim to trace the evolution of eight edge-on star-forming disc galaxies through the analysis of stellar population properties of their thin and thick discs. These galaxies have relatively low stellar masses (4 $\times$ 10$^9$ to 6 $\times$ 10$^{10}$ $M_{\odot}$). We use Multi-Unit Spectroscopic Explorer (MUSE) observations and full-spectrum fitting to produce spatially resolved maps of ages, metallicities and [Mg/Fe]-abundances and extract the star formation histories of stellar discs. Our maps show thick discs that are on average older, more metal-poor and more [Mg/Fe]-enhanced than thin discs. However, age differences between thin and thick discs are small (around 2 Gyr) and the thick discs are younger than previously observed in more massive and more quiescent galaxies. Both thin and thick discs show mostly sub-solar metallicities, and the vertical metallicity gradient is milder than previously observed in similar studies. [Mg/Fe] differences between thick and thin discs are not sharp. The star formation histories of thick discs extend down to recent times, although most of the mass in young stars was formed in thin discs. Our findings show thick discs that are different from old quiescent thick discs previously observed in galaxies of different morphologies and/or different masses. We propose that thick discs in these galaxies did not form quickly at high redshift, but slowly in an extended time. Also, the thin discs formed slowly, however, a larger mass fraction was created at very recent times.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.06268-b31b1b.svg)](https://arxiv.org/abs/2410.06268) | **Euclid: Relativistic effects in the dipole of the 2-point correlation function**  |
|| F. Lepori, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *21 pages, 11 figures, 1 appendix; submitted on behalf of the Euclid Collaboration*|
|**Abstract**|            Gravitational redshift and Doppler effects give rise to an antisymmetric component of the galaxy correlation function when cross-correlating two galaxy populations or two different tracers. In this paper, we assess the detectability of these effects in the Euclid spectroscopic galaxy survey. We model the impact of gravitational redshift on the observed redshift of galaxies in the Flagship mock catalogue using a Navarro-Frenk-White profile for the host haloes. We isolate these relativistic effects, largely subdominant in the standard analysis, by splitting the galaxy catalogue into two populations of faint and bright objects and estimating the dipole of their cross-correlation in four redshift bins. In the simulated catalogue, we detect the dipole signal on scales below $30\,h^{-1}{\rm Mpc}$, with detection significances of $4\,\sigma$ and $3\,\sigma$ in the two lowest redshift bins, respectively. At higher redshifts, the detection significance drops below $2\,\sigma$. Overall, we estimate the total detection significance in the Euclid spectroscopic sample to be approximately $6\,\sigma$. We find that on small scales, the major contribution to the signal comes from the nonlinear gravitational potential. Our study on the Flagship mock catalogue shows that this observable can be detected in Euclid Data Release 2 and beyond.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.06697-b31b1b.svg)](https://arxiv.org/abs/2410.06697) | **JWST-IPA: Chemical Inventory and Spatial Mapping of Ices in the Protostar HOPS370 -- Evidence for an Opacity Hole and Thermal Processing of Ices**  |
|| H. Tyagi, et al. -- incl., <mark>H. Beuther</mark>, <mark>H. Linz</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *Submitted to ApJ. Main text:16 pages with 11 figures*|
|**Abstract**|            The composition of protoplanetary disks, and hence the initial conditions of planet formation, may be strongly influenced by the infall and thermal processing of material during the protostellar phase. Composition of dust and ice in protostellar envelopes, shaped by energetic processes driven by the protostar, serves as the fundamental building material for planets and complex organic molecules. As part of the JWST GO program, "Investigating Protostellar Accretion" (IPA), we observed an intermediate-mass protostar HOPS 370 (OMC2-FIR3) using NIRSpec/IFU and MIRI/MRS. This study presents the gas and ice phase chemical inventory revealed with the JWST in the spectral range of $\sim$2.9 to 28 $\mu$m and explores the spatial variation of volatile ice species in the protostellar envelope. We find evidence for thermal processing of ice species throughout the inner envelope. We present the first high-spatial resolution ($\sim 80$ au) maps of key volatile ice species H$_{2}$O, CO$_{2}$, $^{13}$CO$_2$, CO, and OCN$^-$, which reveal a highly structured and inhomogeneous density distribution of the protostellar envelope, with a deficiency of ice column density that coincides with the jet/outflow shocked knots. Further, we observe high relative crystallinity of H$_{2}$O ice around the shocked knot seen in the H$_2$ and OH wind/outflow, which can be explained by a lack of outer colder material in the envelope along the line of sight due to the irregular structure of the envelope. These observations show clear evidence of thermal processing of the ices in the inner envelope, close to the outflow cavity walls, heated by the luminous protostar.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.06257-b31b1b.svg)](https://arxiv.org/abs/2410.06257) | **UNCOVER: 404 Error -- Models Not Found for the Triply Imaged Little Red Dot A2744-QSO1**  |
|| Y. Ma, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *19 pages, 7 figures, 2 tables. Submitted to ApJ. Comments are welcome*|
|**Abstract**|            JWST has revealed an abundance of compact, red objects at $z\approx5-8$ dubbed "little red dots" (LRDs), whose SEDs display a faint blue UV continuum followed by a steep rise in the optical. Despite extensive study of their characteristic V-shaped SEDs, the nature of LRDs remains unknown. We present a new analysis of the NIRSpec/PRISM spectrum of A2744-QSO1, a triply imaged LRD at $z=7.04$ from the UNCOVER survey. The spectrum shows a strong Balmer break and broad Balmer emission lines, both of which are difficult to explain with models invoking exclusively AGN or stellar contributions. Our fiducial model decomposes the spectrum into a post-starburst galaxy dominating the UV-optical continuum and a reddened AGN being sub-dominant at all wavelength and contributing at $\sim20\%$ level. However, our most credible model infers a stellar mass of $M_\star\approx 4\times10^9\,\mathrm{M_\odot}$ within a radius of $r_\mathrm{e}<30\,$pc, driving its central density to the highest among observations to date. This high central density could be explained if A2744-QSO-1 is the early-forming core of a modern-day massive elliptical galaxy that later puffed up via the inside-out growth channel. The models also necessitate an unusually steep dust law to preserve the strong break strength, though this steepness may be explained by a deficit of large dust grains. It is also probable that these challenges reflect our ignorance of A2744-QSO1's true nature. Future variability and reverberation mapping studies could help disentangle the galaxy and AGN contribution to the continuum, and deeper redder observations could also unveil the dust properties in LRDs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.06730-b31b1b.svg)](https://arxiv.org/abs/2410.06730) | **Systematic collapse of the accretion disc in AGN confirmed by UV photometry and broad line spectra**  |
|| J.-L. Kang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *10 pages, 3 figure, 2 appendices. Submitted to MNRAS. Comments are very welcome!*|
|**Abstract**|            A recent study on the spectral energy distribution (SED) of AGN combined unobscured X-ray sources from the eROSITA eFEDS Survey with high quality optical imaging from Subaru's Hyper Suprime-Cam (HSC). The HSC data enabled accurate host galaxy subtraction as well as giving a uniform black hole mass estimator from the stellar mass. The resulting stacked optical/X-ray SEDs for black holes at fixed mass show a dramatic transition, where the dominating disc component in bright AGN evaporates into an X-ray hot plasma below $L/L_{\rm Edd}\sim 0.01$. The models fit to these datasets predicted the largest change in SED in the rest frame UV ($< 3000\,Å$), but this waveband was not included in the original study. Here we use archival $u$-band and UV photometry to extend the SEDs into this range, and confirm the UV is indeed intrinsically faint in AGN below $L/L_{\rm Edd}\sim 0.01$ as predicted. This dramatic drop in UV photo-ionising flux is also seen from its effect on the broad emission lines. We stack the recently released SDSS DR18 optical spectra for this sample, and show that the broad H$\beta$ line disappears along with the UV bright component at $L/L_{\rm Edd}\sim 0.01$. This shows that there is a population of unobscured, X-ray bright, UV faint AGN which lack broad emission lines (true type 2 Seyferts).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.06836-b31b1b.svg)](https://arxiv.org/abs/2410.06836) | **The warm-hot intergalactic medium (WHIM) in inter-cluster filaments -- A forecast for HUBS observations based on eRASS1 superclusters**  |
|| Y. Zhao, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-10*|
|*Comments*| *submitted to A&A*|
|**Abstract**|            Cosmological simulations indicate that nearly half of the baryons in the nearby Universe are in the warm-hot intergalactic medium (WHIM) phase, with about half of which residing in cosmic filaments. Recent observational studies using stacked survey data and deep exposures of galaxy cluster outskirts have detected soft X-ray excess associated with optically identified filaments. However, the physical characteristics of WHIM in filaments remain largely undetermined due to the lack of direct spectral diagnostics. In this work, we aim to select appropriate targets for WHIM characterization through pointing observations with the future Hot Universe Baryon Surveyor (HUBS) mission, which is designed with eV level energy resolution in the 0.1-2.0 keV band and a 1 square degree field-of-view. We built a sample of 1577 inter-cluster filaments based on the first eROSITA All-Sky Survey (eRASS1) supercluster catalog and estimated their soft X-ray emission, and used their modeled geometrical properties and oxygen line intensities to select four most appropriate candidate targets for HUBS observations. By simulating and analyzing their mock observations, we demonstrated that with 200 ks HUBS exposure for each candidate, the gas properties of individual filaments can be accurately determined, with the temperature constrained to +-0.01 keV, metallicity constrained to < +-0.03 solar, and density constrained to < +-10%. Elemental abundance of O, Ne, Mg, and Fe can be measured separately, providing unprecedented insights into the chemical history of the filament gas. We also showed that direct mapping of the WHIM distribution is promising with narrow-band imaging of the O viii line. Our work forecasts that next-generation X-ray missions such as HUBS will provide substantial improvement in our understanding of the physical status and evolution history of the diffuse WHIM gas in cosmic large-scale structure.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.05376/./figures/fig10.png', 'tmp_2410.05376/./figures/fig4.png', 'tmp_2410.05376/./figures/fig3.png']
copying  tmp_2410.05376/./figures/fig10.png to _build/html/
copying  tmp_2410.05376/./figures/fig4.png to _build/html/
copying  tmp_2410.05376/./figures/fig3.png to _build/html/
exported in  _build/html/2410.05376.md
    + _build/html/tmp_2410.05376/./figures/fig10.png
    + _build/html/tmp_2410.05376/./figures/fig4.png
    + _build/html/tmp_2410.05376/./figures/fig3.png
found figures ['tmp_2410.05397/./co_vs_f770w_pah_all_reso_allmatchedres.png', 'tmp_2410.05397/./co_vs_f770w_pah_all+hii_reso_allmatchedres.png', 'tmp_2410.05397/./co_vs_f770w_pah_all+cendisk_reso_allmatchedres.png', 'tmp_2410.05397/./co_vs_f770w_pah_bygal_reso_allmatchedres.png', 'tmp_2410.05397/./co_at_f770w_pah_0_vs_logmstar.png', 'tmp_2410.05397/./co_at_f770w_pah_0_vs_logssfr.png', 'tmp_2410.05397/./co_vs_f770w_pah_bygal_reso_allmatchedres_normalized.png', 'tmp_2410.05397/./co_vs_f770w_pah_a

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\Sownak}[1]{\textcolor{red}{{\bf #1}}}$
$\newcommand{\saha}[1]{\textcolor{orange}{{\bf #1}}}$
$\newcommand$</div>



<div id="title">

# Quantifying the impact of AGN feedback on the large-scale matter distribution using two- and three-point statistics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.05376-b31b1b.svg)](https://arxiv.org/abs/2410.05376)<mark>Appeared on: 2024-10-10</mark> -  _Accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>B. Saha</mark>, S. Bose

</div>
<div id="abstract">

**Abstract:** Feedback from active galactic nuclei (AGN) plays a critical role in shaping the matter distribution on scales comparable to and larger than individual galaxies. Upcoming surveys such as _Euclid_ and LSST aim to precisely quantify the matter distribution on cosmological scales, making a detailed understanding of AGN feedback effects essential. Hydrodynamical simulations provide an informative framework for studying these effects, in particular by allowing us to vary the parameters that determine the strength of these feedback processes and, consequently, to predict their corresponding impact on the large-scale matter distribution. We use the EAGLE simulations to explore how changes in subgrid viscosity and AGN heating temperature affect the matter distribution, quantified via  2- and 3-point correlation functions, as well as higher order cumulants of the matter distribution. We find that varying viscosity has a small impact ( $\approx 10\%$ ) on scales larger than $1 h^{-1}$ Mpc, while changes to the AGN heating temperature lead to substantial differences, with up to 70 \% variation in gas clustering on small scales ( $\lesssim 1 h^{-1}$ Mpc). By examining the suppression of the power spectrum as a function of time, we identify the redshift range $z = 1.5 - 1$ as a key epoch where AGN feedback begins to dominate in these simulations. The 3-point function provides complementary insight to the more familiar 2-point statistics, and shows more pronounced variations between models on the scale of individual haloes. On the other hand, we find that effects on even larger scales are largely comparable.

</div>

<div id="div_fig1">

<img src="tmp_2410.05376/./figures/fig10.png" alt="Fig20" width="100%"/>

**Figure 20. -** Cumulants of the DM density field relative to gas at different epochs for different smoothing scales. The color of the lines indicates the curve while the line styles indicate the different models in our study. The top panels show the suppression in the cumulants for {\tt AGNdT9}(solid curves) and {\tt AGNdT8}(dashed curves) models. In the bottom panels for completeness, we plot all the models.
     (*fig:fig10*)

</div>
<div id="div_fig2">

<img src="tmp_2410.05376/./figures/fig4.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparing 2pCF of gas with DM between {\tt AGNdT8-9} and {\tt ViscHi-Lo}.The solid lines are for {\tt AGNdT8} and {\tt ViscHi}, and the dashed lines are for {\tt AGNdT9} and {\tt ViscLo}, respectively. The 2pCF is boosted for {\tt AGNdT8} and {\tt ViscHi}, therefore they have more clustering of matter than their counterparts. The solid black line is the 2pCF from the EAGLE reference model at $z=0$, while black dashed line represent the 2pCF at $z=0$ with the AGN feedback turned off. (*fig:fig4*)

</div>
<div id="div_fig3">

<img src="tmp_2410.05376/./figures/fig3.png" alt="Fig2" width="100%"/>

**Figure 2. -** $P(k)$ at different epochs for {\tt AGNdT8} and {\tt ViscHi} model. With evolving time, clustering increases which boosts the $P(k)$. AGN feedback re-distributes matter, which suppresses the $P(k)$. The crossover events, which are departures from general trend of $P(k)$ with $z$, are indicative of the onset of dominance of AGN feedback. Similar behaviour is observed with {\tt AGNdT9} and {\tt ViscLo}, but with different intensity, which is studied later. (*fig:fig3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.05376"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\um}{\mum}$
$\newcommand{\nh}{\ensuremath{N(\mathrm{H})}}$
$\newcommand{\nhtwo}{\ensuremath{N(\mathrm{H_2})}}$
$\newcommand{\htwo}{\ensuremath{\mathrm{H_2}}}$
$\newcommand{\hone}{\ion{H}{1}}$
$\newcommand{\hplus}{\ion{H}{2}}$
$\newcommand{\qpah}{\ensuremath{q_\mathrm{PAH}}}$
$\newcommand{\aco}{\ensuremath{\alpha_\mathrm{CO}}}$
$\newcommand{\inu}{\ensuremath{I_\nu}}$
$\newcommand{\ipah}{\ensuremath{I^\mathrm{PAH}_{\rm F770W}}}$
$\newcommand{\ssfr}{\ensuremath{\mathrm{SFR}/M_\star}}$
$\newcommand{\mstar}{\ensuremath{M_\star}}$
$\newcommand{\norm}{\ensuremath{C_\mathrm{F770W}^\mathrm{PAH}}}$
$\newcommand{\mjysr}{MJy~sr^{-1}}$
$\newcommand{\kkms}{K~km~s^{-1}}$
$\newcommand{\copahunit}{\kkms/(\mjysr)}$
$\newcommand$
$\newcommand{\Ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\UGent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\STScI}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\MPIA}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\AURA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\UCSD}{Department of Astronomy \& Astrophysics, University of California, San Diego, 9500 Gilman Dr., La Jolla, CA 92093, USA}$
$\newcommand{\Whitman}{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\JHU}{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{\OSUPhys}{Department of Physics, The Ohio State University, Columbus, Ohio 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ARI}{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ANU}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\UConn}{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}$
$\newcommand{\UHawaii}{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UniCA}{Université C\^{o}te d'Azur, Observatoire de la C\^{o}te d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\UAlberta}{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}$
$\newcommand{\Arcetri}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\UWyoming}{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LJMU}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\ITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\MPE}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\UMD}{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\UVA}{Department of Astronomy, University of Virginia, Charlottesville, VA, USA}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, Charlottesville, VA, USA}$
$\newcommand{\ASIAA}{Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 106216, Taiwan}$
$\newcommand{\kipac}{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}$
$\newcommand{\aifa}{Argelander-Institut für Astronomie, University of Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\TKU}{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\newcommand{\CarnegieObs}{The Observatories of the Carnegie Institution for Science. 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544, USA}$
$\newcommand{\IAS}{Institute for Advanced Study, 1 Einstein Drive, Princeton, NJ 08540, USA}$
$\newcommand{\COOL}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\ESO}{European Southern Observatory (ESO), Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{\ULyon}{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval, France}$</div>



<div id="title">

# Polycyclic Aromatic Hydrocarbon and CO (2-1) Emission at $50{-}150$ pc Scales in $\nsample$ Nearby Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.05397-b31b1b.svg)](https://arxiv.org/abs/2410.05397)<mark>Appeared on: 2024-10-10</mark> -  _21 pages, 4 figures, 3 tables. Submitted to ApJ_

</div>
<div id="authors">

Ryan~Chown, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Combining Atacama Large Millimeter/sub-millimeter Array CO(2-1) mapping and JWST near- and mid-infrared imaging, we characterize the relationship between CO(2-1) and polycyclic aromatic hydrocarbon (PAH) emission at $\approx 100$ pc resolution in $\nsample$ nearby star-forming galaxies, expanding the sample size from previous $\approx 100$ pc resolution studies by more than an order of magnitude. Focusing on regions of galaxies where most of the gas is likely to be molecular, we find strong correlations between CO(2-1) and 3.3 $\mu$ m, $7.7\mu$ m, and $11.3 \mu$ m PAH emission, estimated from JWST's F335M, F770W, and F1130W filters. We derive power law relations between CO(2-1) and PAH emission, which have indices in the range $0.8{-}1.2$ , implying relatively weak variations in the observed CO-to-PAH ratios across the regions that we study. We find that CO-to-PAH ratios and scaling relationships near $\ion{H}{2}$ regions are similar to those in  diffuse sight lines. The main difference between the two types of regions is that sight lines near $\ion{H}{2}$ regions show higher intensities in all tracers. Galaxy centers, on the other hand, show higher overall intensities and enhanced CO-to-PAH ratios compared to galaxy disks. Individual galaxies show $0.19$ dex scatter in the normalization of CO at fixed $I_{\rm PAH}$ , and this normalization anti-correlates with specific star formation rate ( $\ssfr$ ) and correlates with stellar mass. We provide a prescription that accounts for these galaxy-to-galaxy variations and represents our best current empirical predictor to estimate CO(2-1) intensity from PAH emission, which allows one to take advantage of JWST's excellent sensitivity and resolution to trace cold gas.

</div>

<div id="div_fig1">

<img src="tmp_2410.05397/./co_vs_f770w_pah_all_reso_allmatchedres.png" alt="Fig1.1" width="25%"/><img src="tmp_2410.05397/./co_vs_f770w_pah_all+hii_reso_allmatchedres.png" alt="Fig1.2" width="25%"/><img src="tmp_2410.05397/./co_vs_f770w_pah_all+cendisk_reso_allmatchedres.png" alt="Fig1.3" width="25%"/><img src="tmp_2410.05397/./co_vs_f770w_pah_bygal_reso_allmatchedres.png" alt="Fig1.4" width="25%"/>

**Figure 1. -** **CO(2--1) and starlight continuum-subtracted F770W$_{\rm PAH**$ emission} at $\approx 100$ pc resolution in 66 nearby star-forming galaxies. _Top left:_ All sight lines in our analysis (gray points) with data density contours enclosing the densest 15, 25, 50, 75, and 95\% of the data points. The bins show the median and 16{-}84\% range of CO(2-1) emission in logarithmically-spaced bins of $I_{\rm PAH}^{\rm F770W}$; treating the PAH emission as the independent variable allows us to average the noisier CO(2-1) data. The dashed line shows the best-fit power law describing these binned measurements (Table \ref{tab:fits}).  _Top right:_ As the top left panel but now separately plotting results for sight lines near $\ion${H}{2} regions (yellow, stars) and diffuse emission outside these regions (purple, squares). The two environments show similar CO-to-PAH ratios where they overlap, but the sight lines near $\ion${H}{2} regions show overall brighter intensities. _Bottom left:_ As for the previous figures, but now separating galaxy centers (blue, circles) from disks (green, squares). Galaxy centers show brighter emission and higher CO-to-PAH ratios at the same $I_{\rm F770W}^{\rm PAH}$. _Bottom right:_ Traces show binned results for each individual galaxy. The galaxies show overall similar CO(2-1) vs. F770W$_{\rm PAH}$ relations with moderate offsets from galaxy to galaxy. These offsets correlate with the integrated galaxy properties (see Figure \ref{fig:co_mir_bygal}, with the color bar indicating SFR$/M_\star$). See Table \ref{tab:fits} for ratios and best fits for each panel.
 (*fig:co_mir_all*)

</div>
<div id="div_fig2">

<img src="tmp_2410.05397/./co_at_f770w_pah_0_vs_logmstar.png" alt="Fig2.1" width="25%"/><img src="tmp_2410.05397/./co_at_f770w_pah_0_vs_logssfr.png" alt="Fig2.2" width="25%"/><img src="tmp_2410.05397/./co_vs_f770w_pah_bygal_reso_allmatchedres_normalized.png" alt="Fig2.3" width="25%"/><img src="tmp_2410.05397/./co_vs_f770w_pah_all_reso_allmatchedres_normalized.png" alt="Fig2.4" width="25%"/>

**Figure 2. -** **Galaxy-to-galaxy variations in the CO(2--1) vs. F770W$_{\rm PAH**$ relationship.} CO(2-1) at fixed PAH intensity for individual galaxies (see bottom left panel Fig. \ref{fig:co_mir_all}) as a function of (_left_) galaxy-integrated stellar mass, $M_\star$ and (_right_) specific star formation rate, SFR/$M_\star$. Spearman's $r$, p-value, the best-fit slope, intercept and pivot ($y=m(x-x_0)+b$) to all galaxies, the scatter about the best-fit relation (Eq. \ref{eq:norms}), and vertical scatter are all indicated. We observe a modest correlation between the CO-to-PAH ratio and $M_\star$ and a well-defined anti-correlation between the CO-to-PAH ratio and SFR/$M_\star$. The correlation with $M_\star$ may reflect increased contribution of PAH emission associated with CO-dark gas or atomic gas in low mass galaxies. The anti-correlation with SFR/$M_\star$ likely reflects a mixture of increased radiation field strength, suppression of $X_{\rm CO}$, and enhancement of $R_{21}$ in high SFR/M$_\star$ galaxies, which appear to represent stronger effects than any $q_{\rm PAH}$. Scaled versions of the best-fit normalizations of $I_\mathrm{CO(2-1)}$ vs _WISE_ 12 $\um$ against stellar mass and SFR$/M_\star$ from [Leroy, Bolatto and Sandstrom (2023)]() are shown (dotted lines), showing similar trends.
 (*fig:co_mir_bygal*)

</div>
<div id="div_fig3">

<img src="tmp_2410.05397/./co_vs_f335m_pah_all_reso_allmatchedres.png" alt="Fig3.1" width="50%"/><img src="tmp_2410.05397/./co_vs_f1130w_all_reso_allmatchedres.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** **CO(2--1) intensity as functions of 3.3 $\mu$m and $11.3 \mu$m PAH intensity.** As Figure \ref{fig:co_mir_all}, but now showing PAH intensity captured by (_left_) the F335M filter capturing the 3.3 $\mu$m PAH feature (after continuum subtraction following H. Koziol et al. in preparation) and (_right_) the F1130W filter capturing the 11.3 $\mu$m PAH feature. The relationship between CO(2-1) and these bands resembles that which we observe for F770W$_{\rm PAH}$ in Figure \ref{fig:co_mir_all}, though the specifics of the fits differ (Table \ref{tab:fits}). Of note, the F335M$_{\rm PAH}$ feature has a lower intensity than the others and extracting it from imaging depends critically on stellar continuum subtraction, but the shorter wavelength means that the band offers even higher resolution compared to the other features \citep[see][]{sandstrom2023}.
 (*fig:co_mir_all_otherbands*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.05397"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand\fp{#1}$</div>



<div id="title">

# Relatively young thick discs in low-mass star-forming late-type galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.05761-b31b1b.svg)](https://arxiv.org/abs/2410.05761)<mark>Appeared on: 2024-10-10</mark> -  _20 pages, 9 figures, submitted to A&A_

</div>
<div id="authors">

N. Sattler, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** We aim to trace the evolution of eight edge-on star-forming disc galaxies through the analysis of stellar population properties of their thin and thick discs. These galaxies have relatively low stellar masses (4 $\times$ 10 $^9$ to 6 $\times$ 10 $^{10}$  $M_{\odot}$ ). We use Multi-Unit Spectroscopic Explorer (MUSE) observations and full-spectrum fitting to produce spatially resolved maps of ages, metallicities and [ Mg/Fe ] -abundances and extract the star formation histories of stellar discs. Our maps show thick discs that are on average older, more metal-poor and more [ Mg/Fe ] -enhanced than thin discs.  However, age differences between thin and thick discs are small (around 2 Gyr) and the thick discs are younger than previously observed in more massive and more quiescent galaxies. Both thin and thick discs show mostly sub-solar metallicities, and the vertical metallicity gradient is milder than previously observed in similar studies. [ Mg/Fe ] differences between thick and thin discs are not sharp.  The star formation histories of thick discs extend down to recent times, although most of the mass in young stars was formed in thin discs. Our findings show thick discs that are different from old quiescent thick discs previously observed in galaxies of different morphologies and/or different masses.  We propose that thick discs in these galaxies did not form quickly at high redshift, but slowly in an extended time.  Also, the thin discs formed slowly, however, a larger mass fraction was created at very recent times.

</div>

<div id="div_fig1">

<img src="tmp_2410.05761/./plots/lw_maps.png" alt="Fig3" width="100%"/>

**Figure 3. -** Light-weighted age (left), metallicity (middle) and [Mg/Fe]-abundance (right) maps for the full sample. For the galaxies with distinct morphological thick and thin discs, grey dashed lines mark the regions above and below which the thick disc dominates the vertical surface-brightness profiles. (*fig:lw_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2410.05761/./plots/SFH_maps.png" alt="Fig4" width="100%"/>

**Figure 4. -** Star formation histories in terms of mass fraction per age with evenly spaced age bins every 0.5 Gyr. Uncertainties from Monte Carlo simulations are shown by the shaded areas.
    The left column shows the star formation histories for the full covered regions of the galaxies, while the middle and right columns show the thick and thin disc respectively for the five out of the eight galaxies containing two well-defined disc components (they share the same y-axis with the left column). The additional separate panels on the lower right corner show the star formation histories of the metal-poor regions for ESO 157-49 and IC 1553. (*fig:SFH_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2410.05761/./plots/g_band_images.png" alt="Fig1" width="100%"/>

**Figure 1. -** _g_-band images of the full sample. The image of ESO 157-49 is taken from the Dark Energy Survey DR2
     ([Morganson, Gruendl and Menanteau 2018](), [Abbott, Adamów and Aguena 2021]()) , while the other images are from Pan-STARRS1
     ([Chambers, Magnier and Metcalfe 2016](), [Magnier, Chambers and Flewelling 2020](), [Waters, Magnier and Price 2020](), [Magnier, Sweeney and Chambers 2020](), [Magnier, Schlafly and Finkbeiner 2020](), [Flewelling, Magnier and Chambers 2020]()) . (*fig:g_band*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.05761"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lcdm}{\LambdaCDM}$
$\newcommand{\wcdm}{\wzero\waCDM}$
$\newcommand{\wzero}{w_0}$
$\newcommand{\wa}{w_a}$
$\newcommand{\bn}{{{\bm n}}}$
$\newcommand{\bV}{{\bm V}}$
$\newcommand{\De}{\Delta}$
$\newcommand{\HH}{\mathcal{H}}$
$\newcommand{\al}{\alpha}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\bal}{\boldsymbol{\al}}$
$\newcommand{\bnabla}{\boldsymbol{\nabla}}$
$\newcommand{\ka}{\kappa}$
$\newcommand{\Om}{\Omega}$
$\newcommand{\ds}{{\slash\hspace{-5pt}\partial}}$
$\newcommand{\fnl}{f_{\rm NL}}$
$\newcommand{\GCsp}{\mathrm{GC}_{\rm sp}}$
$\newcommand{\GCph}{\mathrm{GC}_{\rm ph}}$
$\newcommand{\laplaciansphere}{\Delta_{\Omega}}$
$\newcommand{\ns}{n_{\text{s}}}$
$\newcommand{\barnS}{\bar{n}_\text{S}}$
$\newcommand{\rsource}{r_\text{s}}$
$\newcommand{\luminositysource}{L_\text{S}}$
$\newcommand{\fisher}{\tens{F}}$
$\newcommand{\covariance}{\tens{C}}$
$\newcommand{\iu}{{i\mkern1mu}}$
$\newcommand{\std}{\mathrm{std}}$
$\newcommand{\magn}{\ensuremath{\text{magn}}}$
$\newcommand{\dd}{\partial}$
$\newcommand$
$\newcommand{\minus}{\ensuremath{-}}$
$\newcommand{\zmean}{\bar{z}}$
$\newcommand{\MyBlue}{\color[rgb]{0.0, 0.53, 0.74}}$
$\newcommand{\Sangria}{\color[rgb]{0.57, 0.0, 0.04}}$
$\newcommand{\jat}[1]{{\color[rgb]{0,0.6,0}#1}}$
$\newcommand{\jar}[1]{{\color[rgb]{0,0.6,0}{\tt[Julian: #1]}}}$
$\newcommand{\sebt}[1]{{\color[rgb]{1.0,0.3,0.5}#1}}$
$\newcommand{\sebr}[1]{{\color[rgb]{1.0,0.3,0.5}{\tt[Sebastian: #1]}}}$
$\newcommand{\sst}[1]{{\color[rgb]{1.0,0.5,0}#1}}$
$\newcommand{\cam}[1]{\textcolor{red}{{#1}}}$
$\newcommand{\camC}[1]{\textcolor{red}{[{\tt CB: #1}]}}$
$\newcommand{\itt}[1]{\textcolor{purple}{{#1}}}$
$\newcommand{\itc}[1]{\textcolor{purple}{[{\tt IT: #1}]}}$
$\newcommand{\law}[1]{\textcolor{teal}{{#1}}}$
$\newcommand{\lawc}[1]{\textcolor{teal}{[{\tt LD: #1}]}}$
$\newcommand{\orcid}[1]$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\bn}{{{\bm n}}}$
$\newcommand{\bV}{{\bm V}}$
$\newcommand{\De}{\Delta}$
$\newcommand{\HH}{\mathcal{H}}$
$\newcommand{\al}{\alpha}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\bal}{\boldsymbol{\al}}$
$\newcommand{\bnabla}{\boldsymbol{\nabla}}$
$\newcommand{\ga}{\gamma}$
$\newcommand{\ka}{\kappa}$
$\newcommand{\la}{\lambda}$
$\newcommand{\Om}{\Omega}$
$\newcommand{\ds}{{\slash\hspace{-5pt}\partial}}$
$\newcommand{\fnl}{f_{\rm NL}}$
$\newcommand{\GCsp}{\mathrm{GC}_{\rm sp}}$
$\newcommand{\GCph}{\mathrm{GC}_{\rm ph}}$
$\newcommand{\laplaciansphere}{\Delta_{\Omega}}$
$\newcommand{\ns}{n_{\text{s}}}$
$\newcommand{\barnS}{\bar{n}_\text{S}}$
$\newcommand{\rsource}{r_\text{s}}$
$\newcommand{\luminositysource}{L_\text{S}}$
$\newcommand{\fisher}{\tens{F}}$
$\newcommand{\covariance}{\tens{C}}$
$\newcommand{\iu}{{i\mkern1mu}}$
$\newcommand{\std}{\mathrm{std}}$
$\newcommand{\magn}{\ensuremath{\text{magn}}}$
$\newcommand{\dd}{\partial}$
$\newcommand\snr{\ensuremath{\text{S} / \text{N}}}$
$\newcommand{\minus}{\ensuremath{-}}$
$\NewDocumentCommand{\evalat}{sO{\big}mm}{$
$  \IfBooleanTF{#1}{\mleft. #3 \mright|_{#4}}{#3#2|_{#4}}$
$}$
$\newcommand{\zmean}{\bar{z}}$
$\newcommand{\MyBlue}{\color[rgb]{0.0, 0.53, 0.74}}$
$\newcommand{\Sangria}{\color[rgb]{0.57, 0.0, 0.04}}$
$\newcommand\fraC{#1}{{\Sangria[\tt FL: #1]}}$
$\newcommand\fra{#1}{{\Sangria #1}}$
$\newcommand\fraRem{#1}{{\Sangria \sout{#1}}}$
$\newcommand{\jat}[1]{{\color[rgb]{0,0.6,0}#1}}$
$\newcommand{\jar}[1]{{\color[rgb]{0,0.6,0}{\tt[Julian: #1]}}}$
$\newcommand{\sebt}[1]{{\color[rgb]{1.0,0.3,0.5}#1}}$
$\newcommand{\sebr}[1]{{\color[rgb]{1.0,0.3,0.5}{\tt[Sebastian: #1]}}}$
$\newcommand{\sst}[1]{{\color[rgb]{1.0,0.5,0}#1}}$
$\newcommand{\cam}[1]{\textcolor{red}{{#1}}}$
$\newcommand{\camC}[1]{\textcolor{red}{[{\tt CB: #1}]}}$
$\newcommand{\itt}[1]{\textcolor{purple}{{#1}}}$
$\newcommand{\itc}[1]{\textcolor{purple}{[{\tt IT: #1}]}}$
$\newcommand{\law}[1]{\textcolor{teal}{{#1}}}$
$\newcommand{\lawc}[1]{\textcolor{teal}{[{\tt LD: #1}]}}$
$\begin{document}$
$\title{\Euclid: Relativistic effects in the dipole of the 2-point correlation function\thanks{This paper is published on behalf of the Euclid Consortium.}}$
$\newcommand{\orcid}[1] $
$\author{F.~Lepori\thanks{\email{francesca.lepori2@uzh.ch}}\inst{\ref{aff1}}$
$\and S.~Schulz\orcid{0000-0002-8235-9986}\inst{\ref{aff1}}$
$\and I.~Tutusaus\orcid{0000-0002-3199-0399}\inst{\ref{aff2}}$
$\and M.-A.~Breton\inst{\ref{aff3},\ref{aff4},\ref{aff5}}$
$\and S.~Saga\orcid{0000-0002-7387-7570}\inst{\ref{aff6},\ref{aff7}}$
$\and C.~Viglione\inst{\ref{aff8},\ref{aff3}}$
$\and J.~Adamek\orcid{0000-0002-0723-6740}\inst{\ref{aff1}}$
$\and C.~Bonvin\orcid{0000-0002-5318-4064}\inst{\ref{aff9}}$
$\and L.~Dam\orcid{0000-0003-3163-9571}\inst{\ref{aff9}}$
$\and P.~Fosalba\orcid{0000-0002-1510-5214}\inst{\ref{aff8},\ref{aff3}}$
$\and L.~Amendola\orcid{0000-0002-0835-233X}\inst{\ref{aff10}}$
$\and S.~Andreon\orcid{0000-0002-2041-8784}\inst{\ref{aff11}}$
$\and C.~Baccigalupi\orcid{0000-0002-8211-1630}\inst{\ref{aff12},\ref{aff13},\ref{aff14},\ref{aff15}}$
$\and M.~Baldi\orcid{0000-0003-4145-1943}\inst{\ref{aff16},\ref{aff17},\ref{aff18}}$
$\and S.~Bardelli\orcid{0000-0002-8900-0298}\inst{\ref{aff17}}$
$\and D.~Bonino\orcid{0000-0002-3336-9977}\inst{\ref{aff19}}$
$\and E.~Branchini\orcid{0000-0002-0808-6908}\inst{\ref{aff20},\ref{aff21},\ref{aff11}}$
$\and M.~Brescia\orcid{0000-0001-9506-5680}\inst{\ref{aff22},\ref{aff23},\ref{aff24}}$
$\and J.~Brinchmann\orcid{0000-0003-4359-8797}\inst{\ref{aff25},\ref{aff26}}$
$\and A.~Caillat\inst{\ref{aff27}}$
$\and S.~Camera\orcid{0000-0003-3399-3574}\inst{\ref{aff28},\ref{aff29},\ref{aff19}}$
$\and V.~Capobianco\orcid{0000-0002-3309-7692}\inst{\ref{aff19}}$
$\and C.~Carbone\orcid{0000-0003-0125-3563}\inst{\ref{aff30}}$
$\and J.~Carretero\orcid{0000-0002-3130-0204}\inst{\ref{aff31},\ref{aff32}}$
$\and S.~Casas\orcid{0000-0002-4751-5138}\inst{\ref{aff33},\ref{aff34}}$
$\and M.~Castellano\orcid{0000-0001-9875-8263}\inst{\ref{aff35}}$
$\and G.~Castignani\orcid{0000-0001-6831-0687}\inst{\ref{aff17}}$
$\and S.~Cavuoti\orcid{0000-0002-3787-4196}\inst{\ref{aff23},\ref{aff24}}$
$\and A.~Cimatti\inst{\ref{aff36}}$
$\and C.~Colodro-Conde\inst{\ref{aff37}}$
$\and G.~Congedo\orcid{0000-0003-2508-0046}\inst{\ref{aff38}}$
$\and L.~Conversi\orcid{0000-0002-6710-8476}\inst{\ref{aff39},\ref{aff40}}$
$\and Y.~Copin\orcid{0000-0002-5317-7518}\inst{\ref{aff41}}$
$\and F.~Courbin\orcid{0000-0003-0758-6510}\inst{\ref{aff42},\ref{aff43}}$
$\and H.~M.~Courtois\orcid{0000-0003-0509-1776}\inst{\ref{aff44}}$
$\and H.~Degaudenzi\orcid{0000-0002-5887-6799}\inst{\ref{aff45}}$
$\and G.~De~Lucia\orcid{0000-0002-6220-9104}\inst{\ref{aff13}}$
$\and F.~Dubath\orcid{0000-0002-6533-2810}\inst{\ref{aff45}}$
$\and C.~A.~J.~Duncan\orcid{0009-0003-3573-0791}\inst{\ref{aff46}}$
$\and X.~Dupac\inst{\ref{aff40}}$
$\and S.~Dusini\orcid{0000-0002-1128-0664}\inst{\ref{aff47}}$
$\and M.~Farina\orcid{0000-0002-3089-7846}\inst{\ref{aff48}}$
$\and S.~Farrens\orcid{0000-0002-9594-9387}\inst{\ref{aff49}}$
$\and S.~Ferriol\inst{\ref{aff41}}$
$\and M.~Frailis\orcid{0000-0002-7400-2135}\inst{\ref{aff13}}$
$\and E.~Franceschi\orcid{0000-0002-0585-6591}\inst{\ref{aff17}}$
$\and S.~Galeotta\orcid{0000-0002-3748-5115}\inst{\ref{aff13}}$
$\and B.~Gillis\orcid{0000-0002-4478-1270}\inst{\ref{aff38}}$
$\and C.~Giocoli\orcid{0000-0002-9590-7961}\inst{\ref{aff17},\ref{aff18}}$
$\and A.~Grazian\orcid{0000-0002-5688-0663}\inst{\ref{aff50}}$
$\and F.~Grupp\inst{\ref{aff51},\ref{aff52}}$
$\and S.~V.~H.~Haugan\orcid{0000-0001-9648-7260}\inst{\ref{aff53}}$
$\and W.~Holmes\inst{\ref{aff54}}$
$\and F.~Hormuth\inst{\ref{aff55}}$
$\and A.~Hornstrup\orcid{0000-0002-3363-0936}\inst{\ref{aff56},\ref{aff57}}$
$\and S.~Ili\'c\orcid{0000-0003-4285-9086}\inst{\ref{aff58},\ref{aff2}}$
$\and K.~Jahnke\orcid{0000-0003-3804-2137}\inst{\ref{aff59}}$
$\and M.~Jhabvala\inst{\ref{aff60}}$
$\and E.~Keihänen\orcid{0000-0003-1804-7715}\inst{\ref{aff61}}$
$\and A.~Kiessling\orcid{0000-0002-2590-1273}\inst{\ref{aff54}}$
$\and M.~Kilbinger\orcid{0000-0001-9513-7138}\inst{\ref{aff49}}$
$\and B.~Kubik\orcid{0009-0006-5823-4880}\inst{\ref{aff41}}$
$\and M.~Kunz\orcid{0000-0002-3052-7394}\inst{\ref{aff9}}$
$\and H.~Kurki-Suonio\orcid{0000-0002-4618-3063}\inst{\ref{aff62},\ref{aff63}}$
$\and S.~Ligori\orcid{0000-0003-4172-4606}\inst{\ref{aff19}}$
$\and P.~B.~Lilje\orcid{0000-0003-4324-7794}\inst{\ref{aff53}}$
$\and V.~Lindholm\orcid{0000-0003-2317-5471}\inst{\ref{aff62},\ref{aff63}}$
$\and I.~Lloro\orcid{0000-0001-5966-1434}\inst{\ref{aff64}}$
$\and E.~Maiorano\orcid{0000-0003-2593-4355}\inst{\ref{aff17}}$
$\and O.~Mansutti\orcid{0000-0001-5758-4658}\inst{\ref{aff13}}$
$\and O.~Marggraf\orcid{0000-0001-7242-3852}\inst{\ref{aff65}}$
$\and K.~Markovic\orcid{0000-0001-6764-073X}\inst{\ref{aff54}}$
$\and M.~Martinelli\orcid{0000-0002-6943-7732}\inst{\ref{aff35},\ref{aff66}}$
$\and N.~Martinet\orcid{0000-0003-2786-7790}\inst{\ref{aff27}}$
$\and F.~Marulli\orcid{0000-0002-8850-0303}\inst{\ref{aff67},\ref{aff17},\ref{aff18}}$
$\and R.~Massey\orcid{0000-0002-6085-3780}\inst{\ref{aff68}}$
$\and E.~Medinaceli\orcid{0000-0002-4040-7783}\inst{\ref{aff17}}$
$\and M.~Melchior\inst{\ref{aff69}}$
$\and Y.~Mellier\inst{\ref{aff70},\ref{aff71}}$
$\and M.~Meneghetti\orcid{0000-0003-1225-7084}\inst{\ref{aff17},\ref{aff18}}$
$\and E.~Merlin\orcid{0000-0001-6870-8900}\inst{\ref{aff35}}$
$\and G.~Meylan\inst{\ref{aff72}}$
$\and M.~Moresco\orcid{0000-0002-7616-7136}\inst{\ref{aff67},\ref{aff17}}$
$\and L.~Moscardini\orcid{0000-0002-3473-6716}\inst{\ref{aff67},\ref{aff17},\ref{aff18}}$
$\and C.~Neissner\orcid{0000-0001-8524-4968}\inst{\ref{aff73},\ref{aff32}}$
$\and S.-M.~Niemi\inst{\ref{aff74}}$
$\and C.~Padilla\orcid{0000-0001-7951-0166}\inst{\ref{aff73}}$
$\and S.~Paltani\orcid{0000-0002-8108-9179}\inst{\ref{aff45}}$
$\and F.~Pasian\orcid{0000-0002-4869-3227}\inst{\ref{aff13}}$
$\and K.~Pedersen\inst{\ref{aff75}}$
$\and V.~Pettorino\inst{\ref{aff74}}$
$\and S.~Pires\orcid{0000-0002-0249-2104}\inst{\ref{aff49}}$
$\and G.~Polenta\orcid{0000-0003-4067-9196}\inst{\ref{aff76}}$
$\and M.~Poncet\inst{\ref{aff77}}$
$\and L.~A.~Popa\inst{\ref{aff78}}$
$\and F.~Raison\orcid{0000-0002-7819-6918}\inst{\ref{aff51}}$
$\and A.~Renzi\orcid{0000-0001-9856-1970}\inst{\ref{aff79},\ref{aff47}}$
$\and J.~Rhodes\orcid{0000-0002-4485-8549}\inst{\ref{aff54}}$
$\and G.~Riccio\inst{\ref{aff23}}$
$\and E.~Romelli\orcid{0000-0003-3069-9222}\inst{\ref{aff13}}$
$\and M.~Roncarelli\orcid{0000-0001-9587-7822}\inst{\ref{aff17}}$
$\and C.~Rosset\orcid{0000-0003-0286-2192}\inst{\ref{aff80}}$
$\and E.~Rossetti\orcid{0000-0003-0238-4047}\inst{\ref{aff16}}$
$\and R.~Saglia\orcid{0000-0003-0378-7032}\inst{\ref{aff52},\ref{aff51}}$
$\and Z.~Sakr\orcid{0000-0002-4823-3757}\inst{\ref{aff10},\ref{aff2},\ref{aff81}}$
$\and A.~G.~Sánchez\orcid{0000-0003-1198-831X}\inst{\ref{aff51}}$
$\and D.~Sapone\orcid{0000-0001-7089-4503}\inst{\ref{aff82}}$
$\and B.~Sartoris\orcid{0000-0003-1337-5269}\inst{\ref{aff52},\ref{aff13}}$
$\and M.~Schirmer\orcid{0000-0003-2568-9994}\inst{\ref{aff59}}$
$\and P.~Schneider\orcid{0000-0001-8561-2679}\inst{\ref{aff65}}$
$\and T.~Schrabback\orcid{0000-0002-6987-7834}\inst{\ref{aff83}}$
$\and A.~Secroun\orcid{0000-0003-0505-3710}\inst{\ref{aff84}}$
$\and G.~Seidel\orcid{0000-0003-2907-353X}\inst{\ref{aff59}}$
$\and S.~Serrano\orcid{0000-0002-0211-2861}\inst{\ref{aff8},\ref{aff85},\ref{aff3}}$
$\and C.~Sirignano\orcid{0000-0002-0995-7146}\inst{\ref{aff79},\ref{aff47}}$
$\and G.~Sirri\orcid{0000-0003-2626-2853}\inst{\ref{aff18}}$
$\and L.~Stanco\orcid{0000-0002-9706-5104}\inst{\ref{aff47}}$
$\and J.~Steinwagner\orcid{0000-0001-7443-1047}\inst{\ref{aff51}}$
$\and P.~Tallada-Crespí\orcid{0000-0002-1336-8328}\inst{\ref{aff31},\ref{aff32}}$
$\and I.~Tereno\inst{\ref{aff86},\ref{aff87}}$
$\and R.~Toledo-Moreo\orcid{0000-0002-2997-4859}\inst{\ref{aff88}}$
$\and F.~Torranewcommandlot\orcid{0000-0003-1160-1517}\inst{\ref{aff32},\ref{aff31}}$
$\and L.~Valenziano\orcid{0000-0002-1170-0104}\inst{\ref{aff17},\ref{aff89}}$
$\and T.~Vassallo\orcid{0000-0001-6512-6358}\inst{\ref{aff52},\ref{aff13}}$
$\and Y.~Wang\orcid{0000-0002-4749-2984}\inst{\ref{aff90}}$
$\and J.~Weller\orcid{0000-0002-8282-2010}\inst{\ref{aff52},\ref{aff51}}$
$\and E.~Zucca\orcid{0000-0002-5845-8132}\inst{\ref{aff17}}$
$\and C.~Burigana\orcid{0000-0002-3005-5796}\inst{\ref{aff91},\ref{aff89}}$
$\and G.~Fabbian\orcid{0000-0002-3255-4695}\inst{\ref{aff92},\ref{aff93}}$
$\and F.~Finelli\orcid{0000-0002-6694-3269}\inst{\ref{aff17},\ref{aff89}}$
$\and A.~Pezzotta\orcid{0000-0003-0726-2268}\inst{\ref{aff51}}$
$\and V.~Scottez\inst{\ref{aff70},\ref{aff94}}$
$\and M.~Viel\orcid{0000-0002-2642-5707}\inst{\ref{aff12},\ref{aff13},\ref{aff15},\ref{aff14},\ref{aff95}}}$
$\institute{Department of Astrophysics, University of Zurich, Winterthurerstrasse 190, 8057 Zurich, Switzerland\label{aff1}$
$\and$
$Institut de Recherche en Astrophysique et Planétologie (IRAP), Université de Toulouse, CNRS, UPS, CNES, 14 Av. Edouard Belin, 31400 Toulouse, France\label{aff2}$
$\and$
$Institute of Space Sciences (ICE, CSIC), Campus UAB, Carrer de Can Magrans, s/n, 08193 Barcelona, Spain\label{aff3}$
$\and$
$Institut de Ciencies de l'Espai (IEEC-CSIC), Campus UAB, Carrer de Can Magrans, s/n Cerdanyola del Vallés, 08193 Barcelona, Spain\label{aff4}$
$\and$
$Laboratoire Univers et Théorie, Observatoire de Paris, Université PSL, Université Paris Cité, CNRS, 92190 Meudon, France\label{aff5}$
$\and$
$Kobayashi-Maskawa Institute for the Origin of Particles and the Universe, Nagoya University, Chikusa-ku, Nagoya, 464-8602, Japan\label{aff6}$
$\and$
$Institute for Advanced Research, Nagoya University, Chikusa-ku, Nagoya, 464-8601, Japan\label{aff7}$
$\and$
$Institut d'Estudis Espacials de Catalunya (IEEC),  Edifici RDIT, Campus UPC, 08860 Castellnewcommandels, Barcelona, Spain\label{aff8}$
$\and$
$Université de Genève, Département de Physique Théorique and Centre for Astroparticle Physics, 24 quai Ernest-Ansermet, CH-1211 Genève 4, Switzerland\label{aff9}$
$\and$
$Institut für Theoretische Physik, University of Heidelberg, Philosophenweg 16, 69120 Heidelberg, Germany\label{aff10}$
$\and$
$INAF-Osservatorio Astronomico di Brera, Via Brera 28, 20122 Milano, Italy\label{aff11}$
$\and$
$IFPU, Institute for Fundamental Physics of the Universe, via Beirut 2, 34151 Trieste, Italy\label{aff12}$
$\and$
$INAF-Osservatorio Astronomico di Trieste, Via G. B. Tiepolo 11, 34143 Trieste, Italy\label{aff13}$
$\and$
$INFN, Sezione di Trieste, Via Valerio 2, 34127 Trieste TS, Italy\label{aff14}$
$\and$
$SISSA, International School for Advanced Studies, Via Bonomea 265, 34136 Trieste TS, Italy\label{aff15}$
$\and$
$Dipartimento di Fisica e Astronomia, Università di Bologna, Via Gobetti 93/2, 40129 Bologna, Italy\label{aff16}$
$\and$
$INAF-Osservatorio di Astrofisica e Scienza dello Spazio di Bologna, Via Piero Gobetti 93/3, 40129 Bologna, Italy\label{aff17}$
$\and$
$INFN-Sezione di Bologna, Viale Berti Pichat 6/2, 40127 Bologna, Italy\label{aff18}$
$\and$
$INAF-Osservatorio Astrofisico di Torino, Via Osservatorio 20, 10025 Pino Torinese (TO), Italy\label{aff19}$
$\and$
$Dipartimento di Fisica, Università di Genova, Via Dodecaneso 33, 16146, Genova, Italy\label{aff20}$
$\and$
$INFN-Sezione di Genova, Via Dodecaneso 33, 16146, Genova, Italy\label{aff21}$
$\and$
$Department of Physics "E. Pancini", University Federico II, Via Cinthia 6, 80126, Napoli, Italy\label{aff22}$
$\and$
$INAF-Osservatorio Astronomico di Capodimonte, Via Moiariello 16, 80131 Napoli, Italy\label{aff23}$
$\and$
$INFN section of Naples, Via Cinthia 6, 80126, Napoli, Italy\label{aff24}$
$\and$
$Instituto de Astrofísica e Ci\^encias do Espaço, Universidade do Porto, CAUP, Rua das Estrelas, PT4150-762 Porto, Portugal\label{aff25}$
$\and$
$Faculdade de Ci\^encias da Universidade do Porto, Rua do Campo de Alegre, 4150-007 Porto, Portugal\label{aff26}$
$\and$
$Aix-Marseille Université, CNRS, CNES, LAM, Marseille, France\label{aff27}$
$\and$
$Dipartimento di Fisica, Università degli Studi di Torino, Via P. Giuria 1, 10125 Torino, Italy\label{aff28}$
$\and$
$INFN-Sezione di Torino, Via P. Giuria 1, 10125 Torino, Italy\label{aff29}$
$\and$
$INAF-IASF Milano, Via Alfonso Corti 12, 20133 Milano, Italy\label{aff30}$
$\and$
$Centro de Investigaciones Energéticas, Medioambientales y Tecnológicas (CIEMAT), Avenida Complutense 40, 28040 Madrid, Spain\label{aff31}$
$\and$
$Port d'Informació Científica, Campus UAB, C. Albareda s/n, 08193 Bellaterra (Barcelona), Spain\label{aff32}$
$\and$
$Institute for Theoretical Particle Physics and Cosmology (TTK), RWTH Aachen University, 52056 Aachen, Germany\label{aff33}$
$\and$
$Institute of Cosmology and Gravitation, University of Portsmouth, Portsmouth PO1 3FX, UK\label{aff34}$
$\and$
$INAF-Osservatorio Astronomico di Roma, Via Frascati 33, 00078 Monteporzio Catone, Italy\label{aff35}$
$\and$
$Dipartimento di Fisica e Astronomia "Augusto Righi" - Alma Mater Studiorum Università di Bologna, Viale Berti Pichat 6/2, 40127 Bologna, Italy\label{aff36}$
$\and$
$Instituto de Astrofísica de Canarias, Calle Vía Láctea s/n, 38204, San Cristóbal de La Laguna, Tenerife, Spain\label{aff37}$
$\and$
$Institute for Astronomy, University of Edinburgh, Royal Observatory, Blackford Hill, Edinburgh EH9 3HJ, UK\label{aff38}$
$\and$
$European Space Agency/ESRIN, Largo Galileo Galilei 1, 00044 Frascati, Roma, Italy\label{aff39}$
$\and$
$ESAC/ESA, Camino Bajo del Castillo, s/n., Urb. Villafranca del Castillo, 28692 Villanueva de la Cañada, Madrid, Spain\label{aff40}$
$\and$
$Université Claude Bernard Lyon 1, CNRS/IN2P3, IP2I Lyon, UMR 5822, Villeurbanne, F-69100, France\label{aff41}$
$\and$
$Institut de Ciències del Cosmos (ICCUB), Universitat de Barcelona (IEEC-UB), Martí i Franquès 1, 08028 Barcelona, Spain\label{aff42}$
$\and$
$Institució Catalana de Recerca i Estudis Avançats (ICREA), Passeig de Lluís Companys 23, 08010 Barcelona, Spain\label{aff43}$
$\and$
$UCB Lyon 1, CNRS/IN2P3, IUF, IP2I Lyon, 4 rue Enrico Fermi, 69622 Villeurbanne, France\label{aff44}$
$\and$
$Department of Astronomy, University of Geneva, ch. d'Ecogia 16, 1290 Versoix, Switzerland\label{aff45}$
$\and$
$Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK\label{aff46}$
$\and$
$INFN-Padova, Via Marzolo 8, 35131 Padova, Italy\label{aff47}$
$\and$
$INAF-Istituto di Astrofisica e Planetologia Spaziali, via del Fosso del Cavaliere, 100, 00100 Roma, Italy\label{aff48}$
$\and$
$Université Paris-Saclay, Université Paris Cité, CEA, CNRS, AIM, 91191, Gif-sur-Yvette, France\label{aff49}$
$\and$
$INAF-Osservatorio Astronomico di Padova, Via dell'Osservatorio 5, 35122 Padova, Italy\label{aff50}$
$\and$
$Max Planck Institute for Extraterrestrial Physics, Giessenbachstr. 1, 85748 Garching, Germany\label{aff51}$
$\and$
$Universitäts-Sternwarte München, Fakultät für Physik, Ludwig-Maximilians-Universität München, Scheinerstrasse 1, 81679 München, Germany\label{aff52}$
$\and$
$Institute of Theoretical Astrophysics, University of Oslo, P.O. Box 1029 Blindern, 0315 Oslo, Norway\label{aff53}$
$\and$
$Jet Propulsion Laboratory, California Institute of Technology, 4800 Oak Grove Drive, Pasadena, CA, 91109, USA\label{aff54}$
$\and$
$Felix Hormuth Engineering, Goethestr. 17, 69181 Leimen, Germany\label{aff55}$
$\and$
$Technical University of Denmark, Elektrovej 327, 2800 Kgs. Lyngby, Denmark\label{aff56}$
$\and$
$Cosmic Dawn Center (DAWN), Denmark\label{aff57}$
$\and$
$Université Paris-Saclay, CNRS/IN2P3, IJCLab, 91405 Orsay, France\label{aff58}$
$\and$
$Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany\label{aff59}$
$\and$
$NASA Goddard Space Flight Center, Greenbelt, MD 20771, USA\label{aff60}$
$\and$
$Department of Physics and Helsinki Institute of Physics, Gustaf Hällströmin katu 2, 00014 University of Helsinki, Finland\label{aff61}$
$\and$
$Department of Physics, P.O. Box 64, 00014 University of Helsinki, Finland\label{aff62}$
$\and$
$Helsinki Institute of Physics, Gustaf H{ä}llstr{ö}min katu 2, University of Helsinki, Helsinki, Finland\label{aff63}$
$\and$
$NOVA optical infrared instrumentation group at ASTRON, Oude Hoogeveensedijk 4, 7991PD, Dwingeloo, The Netherlands\label{aff64}$
$\and$
$Universität Bonn, Argelander-Institut für Astronomie, Auf dem Hügel 71, 53121 Bonn, Germany\label{aff65}$
$\and$
$INFN-Sezione di Roma, Piazzale Aldo Moro, 2 - c/o Dipartimento di Fisica, Edificio G. Marconi, 00185 Roma, Italy\label{aff66}$
$\and$
$Dipartimento di Fisica e Astronomia "Augusto Righi" - Alma Mater Studiorum Università di Bologna, via Piero Gobetti 93/2, 40129 Bologna, Italy\label{aff67}$
$\and$
$Department of Physics, Institute for Computational Cosmology, Durham University, South Road, DH1 3LE, UK\label{aff68}$
$\and$
$University of Applied Sciences and Arts of Northwestern Switzerland, School of Engineering, 5210 Windisch, Switzerland\label{aff69}$
$\and$
$Institut d'Astrophysique de Paris, 98bis Boulevard Arago, 75014, Paris, France\label{aff70}$
$\and$
$Institut d'Astrophysique de Paris, UMR 7095, CNRS, and Sorbonne Université, 98 bis boulevard Arago, 75014 Paris, France\label{aff71}$
$\and$
$Institute of Physics, Laboratory of Astrophysics, Ecole Polytechnique Fédérale de Lausanne (EPFL), Observatoire de Sauverny, 1290 Versoix, Switzerland\label{aff72}$
$\and$
$Institut de Física d'Altes Energies (IFAE), The Barcelona Institute of Science and Technology, Campus UAB, 08193 Bellaterra (Barcelona), Spain\label{aff73}$
$\and$
$European Space Agency/ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands\label{aff74}$
$\and$
$DARK, Niels Bohr Institute, University of Copenhagen, Jagtvej 155, 2200 Copenhagen, Denmark\label{aff75}$
$\and$
$Space Science Data Center, Italian Space Agency, via del Politecnico snc, 00133 Roma, Italy\label{aff76}$
$\and$
$Centre National d'Etudes Spatiales -- Centre spatial de Toulouse, 18 avenue Edouard Belin, 31401 Toulouse Cedex 9, France\label{aff77}$
$\and$
$Institute of Space Science, Str. Atomistilor, nr. 409 M\u{a}gurele, Ilfov, 077125, Romania\label{aff78}$
$\and$
$Dipartimento di Fisica e Astronomia "G. Galilei", Università di Padova, Via Marzolo 8, 35131 Padova, Italy\label{aff79}$
$\and$
$Université Paris Cité, CNRS, Astroparticule et Cosmologie, 75013 Paris, France\label{aff80}$
$\and$
$Université St Joseph; Faculty of Sciences, Beirut, Lebanon\label{aff81}$
$\and$
$Departamento de Física, FCFM, Universidad de Chile, Blanco Encalada 2008, Santiago, Chile\label{aff82}$
$\and$
$Universität Innsbruck, Institut für Astro- und Teilchenphysik, Technikerstr. 25/8, 6020 Innsbruck, Austria\label{aff83}$
$\and$
$Aix-Marseille Université, CNRS/IN2P3, CPPM, Marseille, France\label{aff84}$
$\and$
$Satlantis, University Science Park, Sede Bld 48940, Leioa-Bilbao, Spain\label{aff85}$
$\and$
$Departamento de Física, Faculdade de Ci\^encias, Universidade de Lisboa, Edifício C8, Campo Grande, PT1749-016 Lisboa, Portugal\label{aff86}$
$\and$
$Instituto de Astrofísica e Ci\^encias do Espaço, Faculdade de Ci\^encias, Universidade de Lisboa, Tapada da Ajuda, 1349-018 Lisboa, Portugal\label{aff87}$
$\and$
$Universidad Politécnica de Cartagena, Departamento de Electrónica y Tecnología de Computadoras,  Plaza del Hospital 1, 30202 Cartagena, Spain\label{aff88}$
$\and$
$INFN-Bologna, Via Irnerio 46, 40126 Bologna, Italy\label{aff89}$
$\and$
$Infrared Processing and Analysis Center, California Institute of Technology, Pasadena, CA 91125, USA\label{aff90}$
$\and$
$INAF, Istituto di Radioastronomia, Via Piero Gobetti 101, 40129 Bologna, Italy\label{aff91}$
$\and$
$Kavli Institute for Cosmology Cambridge, Madingley Road, Cambridge, CB3 0HA, UK\label{aff92}$
$\and$
$Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK\label{aff93}$
$\and$
$Junia, EPA department, 41 Bd Vauban, 59800 Lille, France\label{aff94}$
$\and$
$ICSC - Centro Nazionale di Ricerca in High Performance Computing, Big Data e Quantum Computing, Via Magnanelli 2, Bologna, Italy\label{aff95}}$
$\abstract{$
$Gravitational redshift and Doppler effects give rise to an antisymmetric component of the galaxy correlation function when cross-correlating two galaxy populations or two different tracers. In this paper, we assess the detectability of these effects in the \Euclid spectroscopic galaxy survey. We model the impact of gravitational redshift on the observed redshift of galaxies in the Flagship mock catalogue using a Navarro--Frenk--White profile for the host haloes. We isolate these relativistic effects, largely subdominant in the standard analysis, by splitting the galaxy catalogue into two populations of faint and bright objects and estimating the dipole of their cross-correlation in four redshift bins.$
$In the simulated catalogue, we detect the dipole signal on scales below30 h^{-1}{\rm Mpc}, with detection significances of 4 \sigma and 3 \sigma in the two lowest redshift bins, respectively. At higher redshifts, the detection significance drops below 2 \sigma. Overall, we estimate the total detection significance in the \Euclid spectroscopic sample to be approximately 6 \sigma. We find that on small scales, the major contribution to the signal comes from the nonlinear gravitational potential. Our study on the Flagship mock catalogue shows that this observable can be detected in Euclid Data Release 2 and beyond.$
$    }$
$    \keywords{Cosmology: large-scale structure of Universe -- Gravitation -- Cosmology: theory}$
$   \titlerunning{\Euclid: Relativistic effects in the dipole of the 2-point correlation function}$
$   \authorrunning{F. Lepori et al.}$
$   \maketitle$
$\n\end{document}}\end{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\ga}{\gamma}$
$\newcommand{\la}{\lambda}$
$\newcommand\fraC{#1}$
$\newcommand\fra{#1}$
$\newcommand\fraRem{#1}$</div>



<div id="title">

# $\Euclid$: Relativistic effects in the dipole of the 2-point correlation function$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.06268-b31b1b.svg)](https://arxiv.org/abs/2410.06268)<mark>Appeared on: 2024-10-10</mark> -  _21 pages, 11 figures, 1 appendix; submitted on behalf of the Euclid Collaboration_

</div>
<div id="authors">

F. Lepori, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Gravitational redshift and Doppler effects give rise to an antisymmetric component of the galaxy correlation function when cross-correlating two galaxy populations or two different tracers. In this paper, we assess the detectability of these effects in the $\Euclid$ spectroscopic galaxy survey. We model the impact of gravitational redshift on the observed redshift of galaxies in the Flagship mock catalogue using a Navarro--Frenk--White profile for the host haloes. We isolate these relativistic effects, largely subdominant in the standard analysis, by splitting the galaxy catalogue into two populations of faint and bright objects and estimating the dipole of their cross-correlation in four redshift bins.In the simulated catalogue, we detect the dipole signal on scales below $ 30 h^{-1}{\rm Mpc}$ , with detection significances of $4 \sigma$ and $3 \sigma$ in the two lowest redshift bins, respectively. At higher redshifts, the detection significance drops below $2 \sigma$ . Overall, we estimate the total detection significance in the $\Euclid$ spectroscopic sample to be approximately $6 \sigma$ . We find that on small scales, the major contribution to the signal comes from the nonlinear gravitational potential. Our study on the Flagship mock catalogue shows that this observable can be detected in Euclid Data Release 2 and beyond.

</div>

<div id="div_fig1">

<img src="tmp_2410.06268/./Figures/magn-bias-explain-dens.png" alt="Fig6" width="100%"/>

**Figure 6. -** Correlation between  $F_{\mathrm{H}$\al$pha}$ and $F_{Y}$ for the redshift bin centred at $\bar{z} = 1$ of the Flagship spectroscopic mock catalogue. A continuous line denotes the flux limit of the Euclid Wide Survey, while a dashed line highlights a \YE-band flux cut, splitting the sample in two equally-populated bright and faint populations. The reference fluxes are $F_{\mathrm{H}$\al$pha 0} = 1 {\rm erg} {\rm cm}^{-2} {\rm s}^{-1}$ and $F_{Y0} \equiv 1 {\rm erg} {\rm cm}^{-2} {\rm s}^{-1} {\rm Hz}^{-1}$. The colour bar highlights the density of objects (in arbitrary units).  (*fig:magn-flux*)

</div>
<div id="div_fig2">

<img src="tmp_2410.06268/./Figures/cov_figure.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Theory uncertainty, given by the square root of the diagonal elements of the theory covariance, estimated for the redshift bin $\bar{z} = 1$, with specifics from \cref{tab:spec}, split `$90\%$ faint, $10\%$ bright'. We have assumed a bin size of $\Delta d = 2 h^{-1} {\rm Mpc}$. The top panel shows the different contributions to the uncertainty,
    coming from the pure shot noise term (black dotted line), the purely sample variance term (yellow dashed line), and the cross-term (red dot-dashed line).
    The sum of all contributions is shown as solid blue line. The bottom panel shows the theory uncertainty,
    including all terms, for different sky coverages
    corresponding to the planned data releases of \Euclid and the Flagship mock used in this work.
     (*fig:theory-cov*)

</div>
<div id="div_fig3">

<img src="tmp_2410.06268/./Figures/SN_linear.png" alt="Fig5" width="100%"/>

**Figure 5. -** S/N of the dipole on linear scales ($\geq20 h^{-1} \mathrm{Mpc}$) for a range of population splits. Here the model prediction is given by linear theory. (*fig:sn:linear*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.06268"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

214  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
